In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig
from util import task_to_prompt

In [2]:
with open("prompt-template.txt") as f:
    template = f.read()

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [4]:
peft_model_id = "finetuned-models/Starling-LM-7B-alpha-finetuned"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, 
                                                    quantization_config=bnb_config, 
                                                    return_dict=True, 
                                                    load_in_4bit=True, 
                                                    device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def generate_response(prompt, max_new_tokens=512, temperature=0.5):
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    token_length = len(input_ids["input_ids"][0])
    if token_length > 8192:
        print("Input too long: ", token_length)
        return prompt + "\n\nInput too long. Please try again."
    print("token length:", token_length)
    outputs = model.generate(**input_ids, do_sample=True, temperature=temperature, pad_token_id=tokenizer.eos_token_id, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
from tqdm.notebook import tqdm
import os

for task in tqdm(os.listdir("data/evaluation")):
    if os.path.exists(f"fine_tuned_results/evaluation/{task.replace('.json', '.txt')}"):
        continue
    prompt, output = task_to_prompt("data/evaluation/" + task)
    print(f"{task}", end=" ")
    full_prompt = f"GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant:"
    output = generate_response(full_prompt, max_new_tokens=2048, temperature=0.5)
    with open(f"fine_tuned_results/evaluation/{task.replace('.json', '.txt')}", "w") as f:
        f.write(output)

  0%|          | 0/400 [00:00<?, ?it/s]

c97c0139.json token length: 5775
ea9794b1.json token length: 2667
5d2a5c43.json token length: 1434
fea12743.json token length: 3768
ae58858e.json token length: 2187
0f63c0b9.json token length: 6162


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


e99362f0.json token length: 2556
195ba7dc.json token length: 1422
f3cdc58f.json token length: 2172
a8610ef7.json token length: 1059
84db8fc4.json token length: 2787
4b6b68e5.json token length: 7986
ff72ca3e.json token length: 4173
8ee62060.json token length: 2988
0b17323b.json token length: 3432
c074846d.json token length: 1449
e345f17b.json token length: 759
50aad11f.json token length: 2586
e4075551.json token length: 4662
66e6c45b.json token length: 297
358ba94e.json token length: 4791
d017b73f.json token length: 852
4c177718.json token length: 5082
b7999b51.json token length: 2769
3979b1a8.json token length: 882
bf32578f.json token length: 1296
d19f7514.json token length: 1095
b0722778.json token length: 900
136b0064.json token length: 3105
5b526a93.json token length: 5997
ef26cbf6.json token length: 1212
e633a9e5.json token length: 405
62ab2642.json token length: 1962
73c3b0d8.json token length: 1629
